In [23]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
import datetime
from dateutil import parser
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ast
from networkx import NetworkXError

In [2]:
cluster_output = pd.read_parquet('../data/cluster_output.parquet')
combined_cluster_edges = pd.read_json('../data/combined_cluster_edges.jsonl', lines=True)
news = pd.read_csv('../data/data.csv')
events = pd.read_json('../data/events.json')

In [3]:
news.head()

,Title,Source,Author,Published Date,URL,Country,Content
0,Fed officials warn of inflation risks from tar...,Straits Times,NaN,2025-02-03T23:25:00Z,https://www.straitstimes.com/business/economy/...,Singapore,WASHINGTON - The Trump administration’s plan f...
1,While You Were Sleeping: 5 stories you might h...,Straits Times,NaN,2025-02-03T22:35:49Z,https://www.straitstimes.com/world/while-you-w...,Singapore,"Trump, Sheinbaum reach deal to delay tariffs f..."
2,"Slower growth, souring business sentiment: How...",Channel NewsAsia,Abigail Ng,2025-02-03T22:00:00Z,https://www.channelnewsasia.com/singapore/trum...,Singapore,Analysts say the impact would be cushioned by ...
3,Malaysia's ECRL: A closer look at the US$11.2b...,Channel NewsAsia,Aqil Haziq Mahmud,2025-02-03T22:00:00Z,https://www.channelnewsasia.com/asia/malaysia-...,Singapore,In the second of a four-part series on the Eas...
4,"S&P 500, Nasdaq, pare losses as Trump’s Mexico...",Straits Times,NaN,2025-02-03T21:22:52Z,https://www.straitstimes.com/business/companie...,Singapore,NEW YORK - The major stock indexes closed lowe...


In [4]:
events.head()

,Title,Source,Author,Published Date,URL,Country,Content,FullContent,events
0,Fed officials warn of inflation risks from tar...,Straits Times,None,1738625100000,https://www.straitstimes.com/business/economy/...,Singapore,WASHINGTON - The Trump administration’s plan f...,Fed officials warn of inflation risks from tar...,"[{'event_id': 1, 'event_type': 'Economic Warni..."
1,While You Were Sleeping: 5 stories you might h...,Straits Times,None,1738622149000,https://www.straitstimes.com/world/while-you-w...,Singapore,"Trump, Sheinbaum reach deal to delay tariffs f...",While You Were Sleeping: 5 stories you might h...,"[{'event_id': 1, 'event_type': 'Policy Agreeme..."
2,"Slower growth, souring business sentiment: How...",Channel NewsAsia,Abigail Ng,1738620000000,https://www.channelnewsasia.com/singapore/trum...,Singapore,Analysts say the impact would be cushioned by ...,"Slower growth, souring business sentiment: How...","[{'event_id': 1, 'event_type': 'Economic Polic..."
3,Malaysia's ECRL: A closer look at the US$11.2b...,Channel NewsAsia,Aqil Haziq Mahmud,1738620000000,https://www.channelnewsasia.com/asia/malaysia-...,Singapore,In the second of a four-part series on the Eas...,Malaysia's ECRL: A closer look at the US$11.2b...,"[{'event_id': 1, 'event_type': 'Infrastructure..."
4,"S&P 500, Nasdaq, pare losses as Trump’s Mexico...",Straits Times,None,1738617772000,https://www.straitstimes.com/business/companie...,Singapore,NEW YORK - The major stock indexes closed lowe...,"S&P 500, Nasdaq, pare losses as Trump’s Mexico...","[{'event_id': 1, 'event_type': 'Political Deci..."


In [5]:
events.events.values[0]

[{'event_id': 1,
  'event_type': 'Economic Warning',
  'trigger': 'warn of inflation risks',
  'event_summary': "Federal Reserve officials warned that the Trump administration's trade tariffs pose inflation risks.",
  'arguments': {'agent': 'Federal Reserve officials',
   'object': 'inflation',
   'location': 'United States',
   'time': '2025-02-03',
   'counterpart': 'Trump administration',
   'impact_summary': 'Potential increase in inflation due to new tariffs could affect monetary policy decisions.',
   'cause': 'Trade tariffs introduced by Trump administration'},
  'dependencies': []},
 {'event_id': 2,
  'event_type': 'Trade Policy',
  'trigger': 'slapped tariffs',
  'event_summary': 'President Trump announced new tariffs on the three largest US trade partners, with varying duties on goods from Mexico, Canada, and China.',
  'arguments': {'agent': 'President Trump',
   'object': 'goods from Mexico, Canada, and China',
   'location': 'United States',
   'time': '2025-02-01',
   'co

In [6]:
cluster_output

,event_id,event_type,trigger,event_summary,arguments,dependencies,cluster_id
0,0,Economic Warning,warn of inflation risks,Federal Reserve officials warned that the Trum...,"{'agent': 'Federal Reserve officials', 'cause'...",[],0
1,1,Trade Policy,slapped tariffs,President Trump announced new tariffs on the t...,"{'agent': 'President Trump', 'cause': 'Trump a...",[],219
2,2,Diplomatic Agreement,suspending the tariffs,Trump agreed to suspend tariffs on Mexico and ...,"{'agent': 'President Trump', 'cause': 'Agreeme...",[{'description': 'Suspensions were a response ...,2
3,3,Economic Analysis,will push up inflation and depress growth,Economists predict the new tariffs will increa...,"{'agent': 'Economists', 'cause': 'Implementati...",[{'description': 'Price increases and growth d...,3
4,4,Monetary Policy,held policy rate steady,The Federal Reserve decided to keep interest r...,"{'agent': 'Federal Reserve', 'cause': 'Uncerta...",[{'description': 'Tariff-induced economic unce...,4
...,...,...,...,...,...,...,...
33959,33959,Diplomatic Trip,Blinken: trip to China,US Secretary of State Antony Blinken announced...,"{'agent': 'Antony Blinken', 'cause': 'ongoing ...",[],33954
33960,33960,Rescheduled Diplomatic Visit,rescheduled trip to China,Blinken's trip to China was rescheduled after ...,"{'agent': 'United States', 'cause': 'Chinese s...",[{'description': 'The trip was rescheduled in ...,33960
33961,33961,Spy Incident,detected - and later shot down - a Chinese spy...,The United States detected and shot down a Chi...,"{'agent': 'United States', 'cause': 'Espionage...",[{'description': 'The spy balloon incident dir...,28534
33962,33962,Bilateal Talks,talks in Beijing,Antony Blinken plans bilateral talks in Beijin...,"{'agent': 'Antony Blinken', 'cause': 'Need for...",[{'description': 'Talks in Beijing are related...,33962


In [7]:
combined_cluster_edges.head()

,source_cluster_id,target_cluster_id,relation_type,description,source_trigger,source_summary,target_trigger,target_summary,confidence_score,explanation,evidence_type
0,42,44,causes,The plan to merge USAID into the State Departm...,merging the US international aid agency (USAID...,The Trump administration considers integrating...,90-day pause in most of foreign aid,President Trump issued an executive order paus...,NaN,NaN,NaN
1,447,179,causes,The retaliation is a direct response to the US...,Rubio to make debut in Panama,Marco Rubio embarked on his first internationa...,retaliate against US tariffs,Canada and Mexico have announced that they wil...,NaN,NaN,NaN
2,240,239,causes,South Africa's deadline is a response to press...,blaming Chinese pressure,Taiwan has accused China of exerting pressure ...,deadline to relocate,The South African government set a deadline fo...,NaN,NaN,NaN
3,3973,447,causes,Rubio's trip is in response to Trump's stateme...,surprise threat to retake control of the Panam...,Donald Trump threatens to retake control of th...,Rubio to make debut in Panama,Marco Rubio embarked on his first internationa...,NaN,NaN,NaN
4,1975,1903,causes,Revival of TikTok is a response to its shutdow...,TikTok stopped working,TikTok stopped working for its American users ...,revive TikTok access,President-elect Donald Trump announced plans t...,NaN,NaN,NaN


In [8]:
def create_integrated_knowledge_graph(cluster_output, combined_cluster_edges, news, events):
    """Create an integrated knowledge graph from all data sources"""
    
    # Create a directed graph
    G = nx.DiGraph()
    
    print("Adding cluster events to graph...")
    # Add nodes from cluster_output (event clusters)
    for _, row in cluster_output.iterrows():
        try:
            # Extract event details
            event_id = row['cluster_id']
            event_type = row['event_type']
            event_summary = row['event_summary']
            trigger = row['trigger']
            
            # Parse arguments to get agent (if available)
            try:
                if isinstance(row['arguments'], str):
                    args = eval(row['arguments'])
                else:
                    args = row['arguments']
                
                if isinstance(args, dict):
                    agent = args.get('agent', 'Unknown')
                else:
                    agent = 'Unknown'
            except:
                agent = 'Unknown'
            
            # Add node with attributes
            G.add_node(f"event_{event_id}", 
                       label=f"{event_type}: {trigger}",
                       title=event_summary,
                       group="Event",
                       event_type=event_type,
                       agent=agent,
                       node_type='event_cluster')
        except Exception as e:
            print(f"Error adding cluster node: {e}")
            continue
    
    print("Adding causal edges to graph...")
    # Add edges from combined_cluster_edges
    for _, row in combined_cluster_edges.iterrows():
        try:
            source = row['source_cluster_id']
            target = row['target_cluster_id']
            relation = row['relation_type']
            description = row['description'] if 'description' in row and not pd.isna(row['description']) else ''
            confidence = row['confidence_score'] if 'confidence_score' in row else np.nan
            
            # Only add edges if both nodes exist
            if f"event_{source}" in G.nodes and f"event_{target}" in G.nodes:
                G.add_edge(f"event_{source}", f"event_{target}", 
                          title=f"{relation}: {description}",
                          label=relation,
                          weight=float(confidence) if not pd.isna(confidence) else 0.5,
                          arrows="to",
                          edge_type='causal')
        except Exception as e:
            print(f"Error adding edge: {e}")
            continue
    
    print("Adding news nodes to graph...")
    # Add nodes from news data
    for idx, row in news.iterrows():
        try:
            news_id = f"news_{idx}"
            title = row['Title'] if 'Title' in row and not pd.isna(row['Title']) else f"News {idx}"
            source = row['Source'] if 'Source' in row and not pd.isna(row['Source']) else "Unknown"
            date = row['Published Date'] if 'Published Date' in row and not pd.isna(row['Published Date']) else ""
            url = row['URL'] if 'URL' in row and not pd.isna(row['URL']) else ""
            
            # Add news node
            G.add_node(news_id,
                       label=f"{title[:30]}..." if len(str(title)) > 30 else title,
                       title=f"{title}\nSource: {source}\nDate: {date}\nURL: {url}",
                       group="News",
                       node_type='news',
                       url=url)
        except Exception as e:
            print(f"Error adding news node: {e}")
            continue
    
    print("Extracting events from news articles...")
    # Process the events dataframe
    if 'events' in events.columns:
        for idx, row in events.iterrows():
            try:
                # Get the events column
                events_data = row['events']
                
                # Skip if events data is missing or empty
                if isinstance(events_data, (float, type(None))) and pd.isna(events_data):
                    continue
                    
                # Parse the events data
                if isinstance(events_data, str):
                    try:
                        events_list = ast.literal_eval(events_data)
                    except:
                        print(f"Could not parse events string at index {idx}")
                        continue
                elif isinstance(events_data, list):
                    events_list = events_data
                elif isinstance(events_data, dict):
                    events_list = [events_data]
                else:
                    print(f"Unexpected events data type at index {idx}: {type(events_data)}")
                    continue
                    
                # Process each event
                if isinstance(events_list, list):
                    for event_idx, event in enumerate(events_list):
                        try:
                            if not isinstance(event, dict):
                                continue
                                
                            # Extract event information
                            event_id = event.get('event_id', f"extracted_{idx}_{event_idx}")
                            event_type = event.get('event_type', 'Unknown')
                            trigger = event.get('trigger', 'Unknown')
                            summary = event.get('event_summary', 'No summary available')
                            
                            # Skip if already added from cluster_output
                            if f"event_{event_id}" in G.nodes:
                                continue
                                
                            # Add new event node
                            event_node_id = f"extracted_event_{event_id}"
                            G.add_node(event_node_id,
                                      label=f"{event_type}: {trigger}",
                                      title=summary,
                                      group="Extracted Event",
                                      event_type=event_type,
                                      node_type='extracted_event')
                            
                            # Connect to the news source
                            news_id = f"news_{idx}"
                            if news_id in G.nodes:
                                G.add_edge(news_id, event_node_id,
                                          label="mentioned",
                                          title="This news article mentioned this event",
                                          edge_type='mention')
                            
                            # Add causal relations between events
                            if 'dependencies' in event and event['dependencies']:
                                deps = event['dependencies']
                                if isinstance(deps, list):
                                    for dep in deps:
                                        if isinstance(dep, dict) and 'event_id' in dep:
                                            dep_id = dep['event_id']
                                            rel_type = dep.get('relation_type', 'related')
                                            desc = dep.get('description', 'Related events')
                                            
                                            target_id = f"event_{dep_id}"
                                            if target_id in G.nodes:
                                                G.add_edge(event_node_id, target_id,
                                                         label=rel_type,
                                                         title=desc,
                                                         edge_type='dependency')
                        except Exception as e:
                            print(f"Error processing individual event at index {idx}, event {event_idx}: {e}")
            except Exception as e:
                print(f"Error processing news events at index {idx}: {e}")
    else:
        print("No 'events' column found in events dataframe")
    
    print(f"Knowledge graph created with {len(G.nodes)} nodes and {len(G.edges)} edges")
    return G

def visualize_enhanced_graph(G, filename="us_china_relations_enhanced_graph.html"):
    # Create a Pyvis network
    net = Network(height="800px", width="100%", directed=True, notebook=False)
    
    # Set physics layout options for better visualization
    net.barnes_hut(gravity=-80000, central_gravity=0.3, spring_length=250, spring_strength=0.001, damping=0.09)
    
    # Add nodes and edges
    net.from_nx(G)
    
    # Configure nodes appearance based on node type
    for node in net.nodes:
        # Default size
        node['size'] = 15
        
        # Style based on node type
        if 'node_type' in node:
            node_type = node['node_type']
            
            if node_type == 'news':
                node['shape'] = 'diamond'
                node['size'] = 20
                node['color'] = '#E91E63'  # Pink for news
                
            elif node_type == 'event_cluster':
                node['shape'] = 'dot'
                node['size'] = 25
                
                # Color by event type
                if 'event_type' in node:
                    if 'Economic' in node['event_type']:
                        node['color'] = '#4CAF50'  # Green for economic
                    elif 'Diplomatic' in node['event_type']:
                        node['color'] = '#2196F3'  # Blue for diplomatic
                    elif 'Trade' in node['event_type']:
                        node['color'] = '#FFC107'  # Amber for trade
                    elif 'Spy' in node['event_type'] or 'Incident' in node['event_type']:
                        node['color'] = '#F44336'  # Red for incidents
                    else:
                        node['color'] = '#9C27B0'  # Purple for others
                        
            elif node_type == 'extracted_event':
                node['shape'] = 'triangle'
                node['size'] = 18
                node['color'] = '#FF9800'  # Orange for extracted events
    
    # Configure edges appearance
    for edge in net.edges:
        # Set edge width based on weight
        if 'weight' in edge:
            edge['width'] = edge['weight'] * 5 if not pd.isna(edge['weight']) else 2
        
        # Style edges by type
        if 'edge_type' in edge:
            edge_type = edge['edge_type']
            
            if edge_type == 'causal':
                edge['color'] = '#FF5722'  # Deep orange for causal
                edge['width'] = 3
                
            elif edge_type == 'mention':
                edge['color'] = '#607D8B'  # Blue grey for mentions
                edge['dashes'] = True
                
            elif edge_type == 'dependency':
                edge['color'] = '#8BC34A'  # Light green for dependencies
                edge['arrows'] = 'to'
        
        # Set edge title for hovering
        if 'title' not in edge:
            edge['title'] = edge.get('label', 'Related')

    # Add legends and controls
    net.show_buttons(filter_=['physics'])
    
    # Save the graph
    net.save_graph(filename)
    
    return filename

def create_interactive_timeline(news, events, cluster_output, filename="us_china_timeline_interactive.html"):
    """Create an interactive timeline visualization using Plotly"""
    
    # Format dates properly
    news_data = news.copy()
    
    # Convert dates to datetime objects
    news_data['date'] = pd.to_datetime(news_data['Published Date'], errors='coerce')
    
    # Extract events data
    event_data = []
    for idx, row in events.iterrows():
        if 'events' in row and not pd.isna(row['events']):
            try:
                if isinstance(row['events'], str):
                    event_list = ast.literal_eval(row['events'])
                else:
                    event_list = row['events']
                
                if not isinstance(event_list, list):
                    continue
                    
                for event in event_list:
                    if 'time' in event and event['time']:
                        try:
                            event_date = event['time']
                            
                            # Format date string if needed
                            if not isinstance(event_date, (datetime.date, datetime.datetime)):
                                event_date = parser.parse(event_date)
                                
                            event_data.append({
                                'date': event_date,
                                'title': event.get('trigger', 'Unknown event'),
                                'description': event.get('event_summary', ''),
                                'type': event.get('event_type', 'Unknown'),
                                'agent': event.get('agent', 'Unknown') if isinstance(event.get('arguments', {}), dict) 
                                        else 'Unknown'
                            })
                        except Exception as e:
                            print(f"Error parsing event date: {e}")
            except Exception as e:
                print(f"Error processing events at index {idx}: {e}")
    
    # Convert to dataframe
    event_df = pd.DataFrame(event_data)
    if not event_df.empty:
        event_df['date'] = pd.to_datetime(event_df['date'], errors='coerce')
    
    # Create the timeline figure
    fig = make_subplots(
        rows=2, cols=1,
        specs=[[{"type": "scatter"}], [{"type": "scatter"}]],
        subplot_titles=("News Articles Over Time", "Events Over Time"),
        vertical_spacing=0.1,
        row_heights=[0.5, 0.5]
    )
    
    # Add news articles to timeline
    if not news_data.empty and 'date' in news_data.columns:
        # Group by date and count
        news_counts = news_data.groupby(news_data['date'].dt.date).size().reset_index(name='count')
        news_counts['date'] = pd.to_datetime(news_counts['date'])
        
        # Add to plot
        fig.add_trace(
            go.Scatter(
                x=news_counts['date'],
                y=news_counts['count'],
                mode='lines+markers',
                name='News Articles',
                hovertemplate='%{x}<br>%{y} articles',
                marker=dict(size=8, color='royalblue')
            ),
            row=1, col=1
        )
    
    # Add event timeline
    if not event_df.empty and 'date' in event_df.columns:
        # Create color mapping for event types
        event_types = event_df['type'].unique()
        colors = px.colors.qualitative.Safe
        color_map = {event_type: colors[i % len(colors)] for i, event_type in enumerate(event_types)}
        
        # Group by type and create scatter plot
        for event_type in event_types:
            type_data = event_df[event_df['type'] == event_type]
            
            fig.add_trace(
                go.Scatter(
                    x=type_data['date'],
                    y=[1] * len(type_data),  # All at same level
                    mode='markers',
                    name=event_type,
                    hovertemplate='%{customdata[0]}<br>%{customdata[1]}',
                    customdata=list(zip(type_data['title'], type_data['description'])),
                    marker=dict(
                        size=12,
                        color=color_map.get(event_type, 'gray'),
                        symbol='circle'
                    )
                ),
                row=2, col=1
            )
    
    # Update layout
    fig.update_layout(
        title='US-China Relations: News and Events Timeline',
        height=800,
        template='plotly_white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        hovermode='closest'
    )
    
    # Update y-axis for event timeline
    fig.update_yaxes(
        title_text="", 
        showticklabels=False, 
        showgrid=False,
        row=2, col=1
    )
    
    # Update x-axes
    fig.update_xaxes(
        title_text="Date",
        rangeslider_visible=True,
        row=1, col=1
    )
    fig.update_xaxes(
        title_text="Date",
        rangeslider_visible=True,
        row=2, col=1
    )
    
    # Save to html
    fig.write_html(filename, include_plotlyjs='cdn')
    
    return filename

def analyze_entity_network(cluster_output, combined_cluster_edges, events):
    """Analyze the network to identify key entities and their relationships"""
    
    # Extract entities (agents) from events
    entities = {}
    
    # From cluster_output
    for _, row in cluster_output.iterrows():
        try:
            args = eval(row['arguments']) if isinstance(row['arguments'], str) else row['arguments']
            agent = args.get('agent', 'Unknown')
            
            if agent != 'Unknown':
                if agent not in entities:
                    entities[agent] = {
                        'mentions': 0,
                        'event_types': set(),
                        'connected_entities': set()
                    }
                
                entities[agent]['mentions'] += 1
                entities[agent]['event_types'].add(row['event_type'])
        except:
            pass
    
    # From events dataframe
    for _, row in events.iterrows():
        if 'events' in row and not pd.isna(row['events']):
            try:
                if isinstance(row['events'], str):
                    event_list = ast.literal_eval(row['events'])
                else:
                    event_list = row['events']
                    
                if not isinstance(event_list, list):
                    continue
                    
                for event in event_list:
                    # Get the agent
                    if 'arguments' in event and isinstance(event['arguments'], dict):
                        agent = event['arguments'].get('agent', 'Unknown')
                        counterpart = event.get('counterpart', 'Unknown')
                        
                        # Add agent
                        if agent != 'Unknown':
                            if agent not in entities:
                                entities[agent] = {
                                    'mentions': 0,
                                    'event_types': set(),
                                    'connected_entities': set()
                                }
                                
                            entities[agent]['mentions'] += 1
                            entities[agent]['event_types'].add(event.get('event_type', 'Unknown'))
                            
                            # Add counterpart as connection
                            if counterpart != 'Unknown':
                                entities[agent]['connected_entities'].add(counterpart)
                                
                                # Also add counterpart as entity
                                if counterpart not in entities:
                                    entities[counterpart] = {
                                        'mentions': 0,
                                        'event_types': set(),
                                        'connected_entities': set()
                                    }
                                entities[counterpart]['mentions'] += 1
                                entities[counterpart]['connected_entities'].add(agent)
            except Exception as e:
                print(f"Error analyzing entities: {e}")
    
    # Find connections from causal relationships
    for _, row in combined_cluster_edges.iterrows():
        source_id = row['source_cluster_id']
        target_id = row['target_cluster_id']
        
        # Get agents from source and target clusters
        source_agent = None
        target_agent = None
        
        for _, cluster_row in cluster_output.iterrows():
            if cluster_row['cluster_id'] == source_id:
                try:
                    args = eval(cluster_row['arguments']) if isinstance(cluster_row['arguments'], str) else cluster_row['arguments']
                    source_agent = args.get('agent', None)
                except:
                    pass
                    
            if cluster_row['cluster_id'] == target_id:
                try:
                    args = eval(cluster_row['arguments']) if isinstance(cluster_row['arguments'], str) else cluster_row['arguments']
                    target_agent = args.get('agent', None)
                except:
                    pass
        
        # Record the connection
        if source_agent and target_agent and source_agent != target_agent:
            if source_agent in entities:
                entities[source_agent]['connected_entities'].add(target_agent)
            if target_agent in entities:
                entities[target_agent]['connected_entities'].add(source_agent)
    
    # Convert sets to lists for JSON serialization
    for entity in entities:
        entities[entity]['event_types'] = list(entities[entity]['event_types'])
        entities[entity]['connected_entities'] = list(entities[entity]['connected_entities'])
    
    return entities

def create_entity_network_viz(entities, filename="us_china_entity_network.html"):
    """Create a visualization of the entity network"""
    G = nx.Graph()
    
    # Add nodes for each entity
    for entity, data in entities.items():
        # Skip entities with no connections
        if not data['connected_entities']:
            continue
            
        # Add node with size based on mentions
        size = min(30, max(10, data['mentions'] * 3))
        
        # Determine group color based on most common event type
        if 'event_types' in data and data['event_types']:
            if 'Economic' in data['event_types']:
                group = 'Economic'
            elif 'Diplomatic' in data['event_types']:
                group = 'Diplomatic'
            elif 'Trade' in data['event_types']:
                group = 'Trade'
            elif 'Spy' in data['event_types'] or 'Incident' in data['event_types']:
                group = 'Security'
            else:
                group = 'Other'
        else:
            group = 'Other'
            
        G.add_node(entity, 
                  size=size, 
                  title=f"{entity}\nMentions: {data['mentions']}\nEvent types: {', '.join(data['event_types'])}",
                  group=group)
    
    # Add edges for connections
    for entity, data in entities.items():
        for connection in data['connected_entities']:
            if entity in G.nodes and connection in G.nodes:
                G.add_edge(entity, connection)
    
    # Create visualization
    net = Network(height="800px", width="100%", notebook=False)
    net.barnes_hut(gravity=-80000, central_gravity=0.3, spring_length=200, spring_strength=0.05, damping=0.09)
    net.from_nx(G)
    
    # Style nodes
    for node in net.nodes:
        if 'group' in node:
            if node['group'] == 'Economic':
                node['color'] = '#4CAF50'  # Green
            elif node['group'] == 'Diplomatic':
                node['color'] = '#2196F3'  # Blue
            elif node['group'] == 'Trade':
                node['color'] = '#FFC107'  # Amber
            elif node['group'] == 'Security':
                node['color'] = '#F44336'  # Red
            else:
                node['color'] = '#9C27B0'  # Purple
    
    # Add legend and filters
    net.show_buttons(filter_=['physics'])
    net.save_graph(filename)
    
    return filename


In [9]:
def create_comprehensive_dashboard(cluster_output, combined_cluster_edges, news, events):
    """Create a comprehensive dashboard integrating all visualizations"""
    
    # Create visualizations
    entity_data = analyze_entity_network(cluster_output, combined_cluster_edges, events)
    graph_file = visualize_enhanced_graph(
        create_integrated_knowledge_graph(cluster_output, combined_cluster_edges, news, events)
    )
    timeline_file = create_interactive_timeline(news, events, cluster_output)
    entity_network_file = create_entity_network_viz(entity_data)
    
    # Create the dashboard HTML
    dashboard_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>US-China Relations Analysis Dashboard</title>
        <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1">
        <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/js/bootstrap.bundle.min.js"></script>
        <style>
            body {{ font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; margin: 0; padding: 0; }}
            .tab-content iframe {{ width: 100%; height: 800px; border: none; }}
            .navbar {{ background-color: #343a40; }}
            .nav-link {{ color: white; }}
            .nav-link.active {{ font-weight: bold; background-color: #007bff; }}
            .container-fluid {{ padding: 20px; }}
            h1, h2, h3 {{ color: #343a40; }}
            .card {{ margin-bottom: 20px; }}
            .legend {{ padding: 10px; background-color: #f8f9fa; border-radius: 5px; margin-top: 10px; }}
            .legend-item {{ display: flex; align-items: center; margin-bottom: 5px; }}
            .legend-color {{ width: 20px; height: 20px; border-radius: 50%; margin-right: 8px; }}
            .dashboard-header {{ background-color: #f8f9fa; padding: 20px; margin-bottom: 20px; border-radius: 5px; }}
        </style>
    </head>
    <body>
        <nav class="navbar navbar-expand-lg navbar-dark">
            <div class="container-fluid">
                <a class="navbar-brand" href="#">US-China Relations Analysis</a>
                <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarNav">
                    <span class="navbar-toggler-icon"></span>
                </button>
                <div class="collapse navbar-collapse" id="navbarNav">
                    <ul class="navbar-nav">
                        <li class="nav-item">
                            <a class="nav-link active" data-bs-toggle="pill" href="#dashboard">Dashboard</a>
                        </li>
                        <li class="nav-item">
                            <a class="nav-link" data-bs-toggle="pill" href="#knowledge-graph">Knowledge Graph</a>
                        </li>
                        <li class="nav-item">
                            <a class="nav-link" data-bs-toggle="pill" href="#timeline">Timeline</a>
                        </li>
                        <li class="nav-item">
                            <a class="nav-link" data-bs-toggle="pill" href="#entity-network">Entity Network</a>
                        </li>
                        <li class="nav-item">
                            <a class="nav-link" data-bs-toggle="pill" href="#data-explorer">Data Explorer</a>
                        </li>
                    </ul>
                </div>
            </div>
        </nav>

        <div class="container-fluid">
            <div class="tab-content">
                <!-- Dashboard Overview -->
                <div class="tab-pane fade show active" id="dashboard">
                    <div class="dashboard-header">
                        <h1>US-China Relations Analysis Dashboard</h1>
                        <p class="lead">Explore the complex web of events, causality, and entities in US-China relations through multiple interactive visualizations.</p>
                    </div>
                    
                    <div class="row">
                        <div class="col-md-6">
                            <div class="card">
                                <div class="card-header">
                                    <h3>Knowledge Graph</h3>
                                </div>
                                <div class="card-body">
                                    <p>Visualizes events, news articles, and their causal relationships in a network graph.</p>
                                    <div style="height: 400px; overflow: hidden;">
                                        <iframe src="{graph_file}" style="width: 100%; height: 400px;"></iframe>
                                    </div>
                                    <a href="#knowledge-graph" class="btn btn-primary mt-3" data-bs-toggle="pill">View Full Screen</a>
                                </div>
                            </div>
                        </div>
                        
                        <div class="col-md-6">
                            <div class="card">
                                <div class="card-header">
                                    <h3>Timeline Analysis</h3>
                                </div>
                                <div class="card-body">
                                    <p>Chronological visualization of events and news coverage intensity over time.</p>
                                    <div style="height: 400px; overflow: hidden;">
                                        <iframe src="{timeline_file}" style="width: 100%; height: 400px;"></iframe>
                                    </div>
                                    <a href="#timeline" class="btn btn-primary mt-3" data-bs-toggle="pill">View Full Screen</a>
                                </div>
                            </div>
                        </div>
                    </div>
                    
                    <div class="row mt-4">
                        <div class="col-md-6">
                            <div class="card">
                                <div class="card-header">
                                    <h3>Entity Network</h3>
                                </div>
                                <div class="card-body">
                                    <p>Network of key entities (governments, officials, organizations) and their relationships.</p>
                                    <div style="height: 400px; overflow: hidden;">
                                        <iframe src="{entity_network_file}" style="width: 100%; height: 400px;"></iframe>
                                    </div>
                                    <a href="#entity-network" class="btn btn-primary mt-3" data-bs-toggle="pill">View Full Screen</a>
                                </div>
                            </div>
                        </div>
                        
                        <div class="col-md-6">
                            <div class="card">
                                <div class="card-header">
                                    <h3>Legend & Usage Guide</h3>
                                </div>
                                <div class="card-body">
                                    <h5>Event Types</h5>
                                    <div class="legend">
                                        <div class="legend-item">
                                            <div class="legend-color" style="background-color: #4CAF50;"></div>
                                            <div>Economic Events</div>
                                        </div>
                                        <div class="legend-item">
                                            <div class="legend-color" style="background-color: #2196F3;"></div>
                                            <div>Diplomatic Events</div>
                                        </div>
                                        <div class="legend-item">
                                            <div class="legend-color" style="background-color: #FFC107;"></div>
                                            <div>Trade Policy</div>
                                        </div>
                                        <div class="legend-item">
                                            <div class="legend-color" style="background-color: #F44336;"></div>
                                            <div>Security/Intelligence Incidents</div>
                                        </div>
                                        <div class="legend-item">
                                            <div class="legend-color" style="background-color: #9C27B0;"></div>
                                            <div>Other Events</div>
                                        </div>
                                    </div>
                                    
                                    <h5 class="mt-3">Node Types</h5>
                                    <div class="legend">
                                        <div class="legend-item">
                                            <div class="legend-color" style="background-color: #E91E63;"></div>
                                            <div>News Articles (Diamond Shape)</div>
                                        </div>
                                        <div class="legend-item">
                                            <div class="legend-color" style="background-color: #FF9800;"></div>
                                            <div>Extracted Events (Triangle Shape)</div>
                                        </div>
                                        <div class="legend-item">
                                            <div class="legend-color" style="background-color: #3F51B5;"></div>
                                            <div>Event Clusters (Circle Shape)</div>
                                        </div>
                                    </div>
                                    
                                    <h5 class="mt-3">Usage Tips</h5>
                                    <ul>
                                        <li>Hover over nodes and edges to see detailed information</li>
                                        <li>Use mouse wheel to zoom in/out of visualizations</li>
                                        <li>Drag nodes to rearrange the network graphs</li>
                                        <li>Use the timeline sliders to focus on specific time periods</li>
                                    </ul>
                                </div>
                            </div>
                        </div>
                    </div>
                </div>
                
                <!-- Full Screen Views -->
                <div class="tab-pane fade" id="knowledge-graph">
                    <h2>Knowledge Graph: US-China Relations Events and Causality</h2>
                    <p>This visualization shows events (nodes) extracted from news articles and their causal relationships (edges). Hover over elements for details.</p>
                    <iframe src="{graph_file}"></iframe>
                </div>
                
                <div class="tab-pane fade" id="timeline">
                    <h2>Timeline Analysis: Events and News Coverage Over Time</h2>
                    <p>This visualization shows news volume and significant events over time. Use the sliders to zoom into specific time periods.</p>
                    <iframe src="{timeline_file}"></iframe>
                </div>
                
                <div class="tab-pane fade" id="entity-network">
                    <h2>Entity Network: Key Players in US-China Relations</h2>
                    <p>This visualization shows key entities (governments, officials, organizations) and their relationships based on event co-occurrence.</p>
                    <iframe src="{entity_network_file}"></iframe>
                </div>
                
                <div class="tab-pane fade" id="data-explorer">
                    <h2>Data Explorer: Raw Data Tables</h2>
                    
                    <div class="card mb-4">
                        <div class="card-header">
                            <h3>Cluster Output</h3>
                        </div>
                        <div class="card-body">
                            <div style="height: 400px; overflow: auto;">
                                {cluster_output.to_html(classes="table table-striped table-hover")}
                            </div>
                        </div>
                    </div>
                    
                    <div class="card mb-4">
                        <div class="card-header">
                            <h3>Causal Relationships</h3>
                        </div>
                        <div class="card-body">
                            <div style="height: 400px; overflow: auto;">
                                {combined_cluster_edges.to_html(classes="table table-striped table-hover")}
                            </div>
                        </div>
                    </div>
                    
                    <div class="card">
                        <div class="card-header">
                            <h3>News Articles</h3>
                        </div>
                        <div class="card-body">
                            <div style="height: 400px; overflow: auto;">
                                {news.head(100).to_html(classes="table table-striped table-hover")}
                            </div>
                        </div>
                    </div>
                </div>
            </div>
        </div>
        
        <footer class="bg-dark text-white text-center p-3 mt-5">
            <p>US-China Relations Knowledge Graph Analysis Dashboard</p>
        </footer>
        
        <script>
            // JavaScript to handle tab switching
            document.addEventListener('DOMContentLoaded', function() {{
                var pills = document.querySelectorAll('.nav-link');
                pills.forEach(function(pill) {{
                    pill.addEventListener('click', function(e) {{
                        e.preventDefault();
                        
                        // Remove active class from all pills
                        pills.forEach(function(p) {{
                            p.classList.remove('active');
                        }});
                        
                        // Add active class to clicked pill
                        this.classList.add('active');
                        
                        // Get target tab
                        var target = this.getAttribute('href').replace('#', '');
                        
                        // Hide all tab contents
                        var tabContents = document.querySelectorAll('.tab-pane');
                        tabContents.forEach(function(tab) {{
                            tab.classList.remove('show', 'active');
                        }});
                        
                        // Show target tab
                        document.getElementById(target).classList.add('show', 'active');
                    }});
                }});
            }});
        </script>
    </body>
    </html>
    """
    
    with open("us_china_dashboard.html", 'w') as f:
        f.write(dashboard_html)
    
    return "us_china_dashboard.html"



In [10]:
def parse_events_data(events_df):
    """Helper function to parse and clean events data"""
    parsed_events = []
    
    if 'events' in events_df.columns:
        for idx, row in events_df.iterrows():
            if pd.isna(row['events']):
                continue
                
            try:
                # Handle different formats of events data
                if isinstance(row['events'], str):
                    if row['events'].startswith("[{") or row['events'].startswith("{"):
                        events_list = ast.literal_eval(row['events'])
                    else:
                        # Skip if not in expected format
                        continue
                elif isinstance(row['events'], (list, dict)):
                    events_list = row['events']
                else:
                    continue
                    
                # Convert to list if it's a dict
                if isinstance(events_list, dict):
                    events_list = [events_list]
                    
                for event in events_list:
                    if not isinstance(event, dict):
                        continue
                        
                    # Extract key information
                    event_id = event.get('event_id', None)
                    event_type = event.get('event_type', 'Unknown')
                    trigger = event.get('trigger', 'Unknown')
                    summary = event.get('event_summary', '')
                    
                    # Extract arguments
                    if 'arguments' in event:
                        args = event['arguments']
                        if isinstance(args, str):
                            try:
                                args = ast.literal_eval(args)
                            except:
                                args = {'agent': 'Unknown'}
                    else:
                        args = {'agent': 'Unknown'}
                    
                    # Create cleaned event entry
                    parsed_events.append({
                        'event_id': event_id,
                        'event_type': event_type,
                        'trigger': trigger,
                        'event_summary': summary,
                        'arguments': args,
                        'news_id': idx
                    })
            except Exception as e:
                print(f"Error parsing event at index {idx}: {e}")
    
    return pd.DataFrame(parsed_events)

In [11]:
def prune_edges(combined_cluster_edges_df, threshold=0.7):
    """Prune edges based on confidence score and significance"""
    # Keep only high-confidence edges
    if 'confidence_score' in combined_cluster_edges_df.columns:
        pruned_edges = combined_cluster_edges_df[
            combined_cluster_edges_df['confidence_score'] >= threshold
        ].copy()
    else:
        pruned_edges = combined_cluster_edges_df.copy()
    
    return pruned_edges

In [12]:
def cluster_events_by_topic(cluster_output_df, n_topics=10):
    """Group events into topics using text similarity"""
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.cluster import KMeans
    
    # Create TF-IDF vectors from event summaries
    vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(cluster_output_df['event_summary'])
    
    # Cluster events
    kmeans = KMeans(n_clusters=n_topics, random_state=42)
    cluster_output_df['topic_cluster'] = kmeans.fit_predict(tfidf_matrix)
    
    return cluster_output_df

In [13]:
def create_temporal_windows(events_df, window_size='3M'):
    """Create temporal windows to analyze events in smaller time periods"""
    events_df['date'] = pd.to_datetime(events_df['Published Date'])
    events_df['time_window'] = events_df['date'].dt.to_period(window_size)
    return events_df

In [14]:
def create_hierarchical_graph(cluster_output_df, combined_cluster_edges_df):
    """Create a hierarchical graph structure with multiple levels of detail"""
    # Level 1: Topic clusters
    topic_clusters = cluster_events_by_topic(cluster_output_df)
    
    # Level 2: Important events within each topic
    def get_important_events(group):
        # Select events with most connections or highest impact
        return group.nlargest(5, 'num_connections')
    
    topic_clusters['num_connections'] = topic_clusters['event_id'].map(
        combined_cluster_edges_df['source_cluster_id'].value_counts()
    ).fillna(0)
    
    important_events = topic_clusters.groupby('topic_cluster').apply(get_important_events)
    
    return topic_clusters, important_events

In [44]:
# def create_filtered_graph(G, topic_clusters, important_events, 
#                         max_nodes=1000,
#                         min_connections=2,
#                         show_only_important=True):
#     """Create an interactive graph with filtering options and initial filtering applied"""
    
#     # Create a subgraph with filtered nodes
#     is_directed = G.is_directed()
#     filtered_G = G.copy()
#     nodes_to_remove = []
    
#     # First, remove all news nodes and their mentioned edges
#     for node, attrs in filtered_G.nodes(data=True):
#         if attrs.get('node_type') == 'news':
#             nodes_to_remove.append(node)
    
#     # Remove news nodes
#     filtered_G.remove_nodes_from(nodes_to_remove)
#     nodes_to_remove = []  # Reset list for next filtering step
    
#     # Now identify event clusters and their relationships
#     event_clusters = {}
#     for node, attrs in filtered_G.nodes(data=True):
#         if isinstance(node, str) and node.startswith('event_'):
#             try:
#                 event_id = int(node.split('_')[1])
#                 if event_id in topic_clusters.index:
#                     topic = topic_clusters.loc[event_id, 'topic_cluster']
#                     event_clusters[node] = topic
#             except (ValueError, KeyError):
#                 continue
    
#     # Keep only event nodes with meaningful connections
#     for node, attrs in filtered_G.nodes(data=True):
#         try:
#             neighbors = list(filtered_G.neighbors(node))
            
#             # Only check event nodes now
#             if attrs.get('node_type') == 'event_cluster':
#                 has_important_edge = False
#                 for neighbor in neighbors:
#                     if neighbor in filtered_G:
#                         edge_data = filtered_G.get_edge_data(node, neighbor)
#                         if edge_data and edge_data.get('edge_type') in ['causal', 'happens_before']:
#                             has_important_edge = True
#                             break
#                 if not has_important_edge and not (node in event_clusters and len(neighbors) > 2):
#                     nodes_to_remove.append(node)
#         except NetworkXError:
#             nodes_to_remove.append(node)
#             continue
    
#     # Remove filtered nodes
#     filtered_G.remove_nodes_from(nodes_to_remove)
    
#     # Create the final graph with the same type as the input graph
#     final_G = nx.DiGraph() if is_directed else nx.Graph()
    
#     # Add nodes and edges directly, excluding mentioned relationships
#     for node, attrs in filtered_G.nodes(data=True):
#         final_G.add_node(node, **attrs)
    
#     for u, v, attrs in filtered_G.edges(data=True):
#         if attrs.get('edge_type') in ['causal', 'happens_before']:  # Only add these types
#             final_G.add_edge(u, v, **attrs)

#     print(f"\nFiltered graph statistics:")
#     print(f"Nodes remaining: {final_G.number_of_nodes()} (from {G.number_of_nodes()})")
#     print(f"Edges remaining: {final_G.number_of_edges()} (from {G.number_of_edges()})")
#     print(f"Topics represented: {len(set(event_clusters.values()))}")
    
#     # Create the visualization
#     net = Network(height="800px", width="100%", directed=is_directed, notebook=False)
    
#     net.html = """
#     <div style="position: absolute; top: 10px; right: 10px; z-index: 999; background: white; padding: 10px; border: 1px solid #ccc;">
#         <h3>Network Controls</h3>
#         <button onclick="network.stabilize()">Stabilize Layout</button>
#         <button onclick="network.fit()">Fit View</button>
#         <div style="margin-top: 10px;">
#             <label>Physics: </label>
#             <button onclick="togglePhysics()">Toggle Physics</button>
#         </div>
#         <div style="margin-top: 10px;">
#             <label>Node Legend:</label>
#             <div style="display: flex; flex-direction: column; margin-top: 5px;">
#                 <div><span style="display: inline-block; width: 15px; height: 15px; background: #E91E63; border-radius: 0; transform: rotate(45deg);"></span> News Articles</div>
#                 <div><span style="display: inline-block; width: 15px; height: 15px; background: #4CAF50; border-radius: 50%;"></span> Economic Events</div>
#                 <div><span style="display: inline-block; width: 15px; height: 15px; background: #2196F3; border-radius: 50%;"></span> Diplomatic Events</div>
#                 <div><span style="display: inline-block; width: 15px; height: 15px; background: #FFC107; border-radius: 50%;"></span> Trade Events</div>
#                 <div><span style="display: inline-block; width: 15px; height: 15px; background: #F44336; border-radius: 50%;"></span> Security Events</div>
#                 <div><span style="display: inline-block; width: 15px; height: 15px; background: #9C27B0; border-radius: 50%;"></span> Other Events</div>
#             </div>
#         </div>
#     </div>

#     <script>
#     function togglePhysics() {
#         const physics = !network.physics.options.enabled;
#         network.physics.options.enabled = physics;
#         if (physics) {
#             network.startSimulation();
#         } else {
#             network.stopSimulation();
#         }
#     }
#     </script>
#     """ + net.html
    
#     # Use force-directed layout for better distribution
#     net.options.physics.solver = "forceAtlas2Based"
#     net.options.physics.forceAtlas2Based = {
#         "gravitationalConstant": -100,
#         "centralGravity": 0.01,
#         "springLength": 200,
#         "springConstant": 0.08,
#         "damping": 0.4,
#         "avoidOverlap": 0.8
#     }
#     net.options.physics.stabilization = {
#         "enabled": True,
#         "iterations": 1000
#     }
#     net.options.interaction = {
#         "navigationButtons": True,
#         "keyboard": True
#     }
    
#     net.from_nx(final_G)
    
#     # Add this section for better labels and styling
#     for node in net.nodes:
#         if 'node_type' in node:
#             node_type = node['node_type']
#             if node_type == 'news':
#                 node['shape'] = 'diamond'
#                 node['size'] = 20
#                 node['color'] = '#E91E63'
#                 # Show article title instead of ID
#                 if 'title' in node:
#                     node['label'] = node['title'][:50] + '...' if len(node['title']) > 50 else node['title']
#                     node['title'] = f"Source: {node.get('source', 'Unknown')}\nDate: {node.get('date', 'Unknown')}\n\n{node.get('title', '')}"
            
#             elif node_type == 'event_cluster':
#                 node['shape'] = 'dot'
#                 node['size'] = 25
#                 # Show main event description instead of ID
#                 if 'main_event' in node:
#                     node['label'] = node['main_event'][:50] + '...' if len(node['main_event']) > 50 else node['main_event']
#                     node['title'] = f"Event Type: {node.get('event_type', 'Unknown')}\nDate: {node.get('date', 'Unknown')}\n\nDescription: {node.get('main_event', '')}"
                
#                 if 'event_type' in node:
#                     if 'Economic' in node['event_type']:
#                         node['color'] = '#4CAF50'
#                     elif 'Diplomatic' in node['event_type']:
#                         node['color'] = '#2196F3'
#                     elif 'Trade' in node['event_type']:
#                         node['color'] = '#FFC107'
#                     elif 'Spy' in node['event_type'] or 'Incident' in node['event_type']:
#                         node['color'] = '#F44336'
#                     else:
#                         node['color'] = '#9C27B0'

#         # Make labels more readable
#         node['font'] = {'size': 14, 'color': 'black', 'face': 'arial'}
#         node['labelHighlightBold'] = True

#     # Style edges with meaningful labels
#     for edge in net.edges:
#         # Set default values first
#         edge['width'] = 2
#         edge['color'] = '#9E9E9E'  # Default gray color
#         edge['label'] = ''
        
#         if 'edge_type' in edge:
#             edge_type = edge['edge_type']
#             if edge_type == 'causal':
#                 edge['label'] = 'causes'
#                 edge['color'] = '#2196F3'
#             elif edge_type == 'happens_before':
#                 edge['label'] = 'precedes'
#                 edge['color'] = '#9E9E9E'
        
#         edge['font'] = {'size': 10, 'color': edge['color']}

    
#     return net

def create_filtered_graph(G, topic_clusters, important_events, 
                        max_nodes=1000,
                        min_connections=2,
                        show_only_important=True):
    """Create an interactive graph with filtering options and analytical features"""
    
    # Create a subgraph with filtered nodes
    is_directed = G.is_directed()
    filtered_G = G.copy()
    nodes_to_remove = []
    
    # First, remove all news nodes and their mentioned edges
    for node, attrs in filtered_G.nodes(data=True):
        if attrs.get('node_type') == 'news':
            nodes_to_remove.append(node)
    
    # Remove news nodes
    filtered_G.remove_nodes_from(nodes_to_remove)
    nodes_to_remove = []  # Reset list for next filtering step
    
    # Now identify event clusters and their relationships
    event_clusters = {}
    for node, attrs in filtered_G.nodes(data=True):
        if isinstance(node, str) and node.startswith('event_'):
            try:
                event_id = int(node.split('_')[1])
                if event_id in topic_clusters.index:
                    topic = topic_clusters.loc[event_id, 'topic_cluster']
                    event_clusters[node] = topic
            except (ValueError, KeyError):
                continue
    
    # Keep only event nodes with meaningful connections
    for node, attrs in filtered_G.nodes(data=True):
        try:
            neighbors = list(filtered_G.neighbors(node))
            
            # Only check event nodes now
            if attrs.get('node_type') == 'event_cluster':
                has_important_edge = False
                for neighbor in neighbors:
                    if neighbor in filtered_G:
                        edge_data = filtered_G.get_edge_data(node, neighbor)
                        if edge_data and edge_data.get('edge_type') in ['causal', 'happens_before']:
                            has_important_edge = True
                            break
                if not has_important_edge and not (node in event_clusters and len(neighbors) > 2):
                    nodes_to_remove.append(node)
        except NetworkXError:
            nodes_to_remove.append(node)
            continue
    
    # Remove filtered nodes
    filtered_G.remove_nodes_from(nodes_to_remove)
    
    # Create the final graph with the same type as the input graph
    final_G = nx.DiGraph() if is_directed else nx.Graph()
    
    # Add nodes and edges directly, excluding mentioned relationships
    for node, attrs in filtered_G.nodes(data=True):
        final_G.add_node(node, **attrs)
    
    for u, v, attrs in filtered_G.edges(data=True):
        if attrs.get('edge_type') in ['causal', 'happens_before']:  # Only add these types
            final_G.add_edge(u, v, **attrs)

    print(f"\nFiltered graph statistics:")
    print(f"Nodes remaining: {final_G.number_of_nodes()} (from {G.number_of_nodes()})")
    print(f"Edges remaining: {final_G.number_of_edges()} (from {G.number_of_edges()})")
    print(f"Topics represented: {len(set(event_clusters.values()))}")
    
    # Create the visualization
    net = Network(height="800px", width="100%", directed=is_directed, notebook=False)
    
    # Use force-directed layout for better distribution
    net.options.physics.solver = "forceAtlas2Based"
    net.options.physics.forceAtlas2Based = {
        "gravitationalConstant": -100,
        "centralGravity": 0.01,
        "springLength": 200,
        "springConstant": 0.08,
        "damping": 0.4,
        "avoidOverlap": 0.8
    }
    net.options.physics.stabilization = {
        "enabled": True,
        "iterations": 1000
    }
    net.options.interaction = {
        "navigationButtons": True,
        "keyboard": True,
        "hover": True,
        "multiselect": True,
        "tooltipDelay": 300
    }
    
    net.from_nx(final_G)
    
    # Style nodes with analytical features
    for node in net.nodes:
        if 'node_type' in node:
            node_type = node['node_type']
            if node_type == 'event_cluster':
                # Calculate node importance based on connections and their types
                neighbors = list(final_G.neighbors(node['id']))
                causal_connections = sum(1 for n in neighbors 
                                      if final_G.get_edge_data(node['id'], n).get('edge_type') == 'causal')
                temporal_connections = sum(1 for n in neighbors 
                                        if final_G.get_edge_data(node['id'], n).get('edge_type') == 'happens_before')
                
                # Make nodes with causal relationships larger
                importance = causal_connections * 2 + temporal_connections
                node['size'] = min(50, max(25, 25 + importance * 5))
                
                node['shape'] = 'dot'
                
                # Show main event description and analytical info
                if 'main_event' in node:
                    node['label'] = node['main_event'][:50] + '...'
                    node['title'] = f"""
Event: {node['main_event']}
Type: {node.get('event_type', 'Unknown')}
Impact: {causal_connections} causal connections
Timeline: {temporal_connections} temporal connections
Date: {node.get('date', 'Unknown')}
                    """
                
                if 'event_type' in node:
                    if 'Economic' in node['event_type']:
                        node['color'] = '#4CAF50'
                    elif 'Diplomatic' in node['event_type']:
                        node['color'] = '#2196F3'
                    elif 'Trade' in node['event_type']:
                        node['color'] = '#FFC107'
                    elif 'Spy' in node['event_type'] or 'Incident' in node['event_type']:
                        node['color'] = '#F44336'
                    else:
                        node['color'] = '#9C27B0'

                # Make labels more readable
                node['font'] = {'size': 14, 'color': 'black', 'face': 'arial'}
                node['labelHighlightBold'] = True

    # Style edges to emphasize relationships
    for edge in net.edges:
        if 'edge_type' in edge:
            edge_type = edge['edge_type']
            if edge_type == 'causal':
                edge['color'] = '#2196F3'
                edge['width'] = 4  # Make causal relationships more prominent
                edge['label'] = 'causes'
                edge['arrows'] = 'to'  # Add directional arrows
            elif edge_type == 'happens_before':
                edge['color'] = '#9E9E9E'
                edge['width'] = 2
                edge['label'] = 'precedes'
                edge['dashes'] = True  # Use dashed lines for temporal relationships
            
            edge['font'] = {'size': 10, 'color': edge['color'], 'align': 'middle'}
            edge['smooth'] = {'type': 'curvedCW', 'roundness': 0.2}

    # Add analytical legend and controls
    net.html = """
    <div style="position: absolute; top: 10px; right: 10px; z-index: 999; background: white; padding: 10px; border: 1px solid #ccc;">
        <h3>Event Analysis</h3>
        <div style="margin-top: 10px;">
            <h4>Key Insights:</h4>
            <ul>
                <li>Larger nodes = More influential events (more causal relationships)</li>
                <li>Solid blue arrows = Direct cause-effect relationships</li>
                <li>Dashed gray lines = Temporal sequence</li>
                <li>Node colors = Event categories</li>
            </ul>
        </div>
        <div style="margin-top: 10px;">
            <h4>Event Categories:</h4>
            <div style="display: flex; flex-direction: column; gap: 5px;">
                <div><span style="display: inline-block; width: 15px; height: 15px; background: #4CAF50; border-radius: 50%;"></span> Economic Events</div>
                <div><span style="display: inline-block; width: 15px; height: 15px; background: #2196F3; border-radius: 50%;"></span> Diplomatic Events</div>
                <div><span style="display: inline-block; width: 15px; height: 15px; background: #FFC107; border-radius: 50%;"></span> Trade Events</div>
                <div><span style="display: inline-block; width: 15px; height: 15px; background: #F44336; border-radius: 50%;"></span> Security Events</div>
                <div><span style="display: inline-block; width: 15px; height: 15px; background: #9C27B0; border-radius: 50%;"></span> Other Events</div>
            </div>
        </div>
        <div style="margin-top: 20px;">
            <button onclick="network.fit()">Fit View</button>
            <button onclick="togglePhysics()">Toggle Physics</button>
        </div>
        <div style="margin-top: 10px;">
            <small>Hover over nodes for detailed information</small>
        </div>
    </div>
    <script>
    function togglePhysics() {
        const physics = !network.physics.options.enabled;
        network.physics.options.enabled = physics;
        if (physics) {
            network.startSimulation();
        } else {
            network.stopSimulation();
        }
    }
    </script>
    """ + net.html
    
    return net


In [45]:
def analyze_network_statistics(G):
    """Print detailed statistics about the network"""
    
    # Basic statistics
    print("Network Statistics:")
    print(f"Total nodes: {G.number_of_nodes()}")
    print(f"Total edges: {G.number_of_edges()}")
    
    # Node type breakdown
    node_types = {}
    for node, attrs in G.nodes(data=True):
        node_type = attrs.get('node_type', 'unknown')
        node_types[node_type] = node_types.get(node_type, 0) + 1
    
    print("\nNode Types:")
    for node_type, count in node_types.items():
        print(f"- {node_type}: {count} nodes")
    
    # Edge type breakdown
    edge_types = {}
    for _, _, attrs in G.edges(data=True):
        edge_type = attrs.get('edge_type', 'unknown')
        edge_types[edge_type] = edge_types.get(edge_type, 0) + 1
    
    print("\nEdge Types:")
    for edge_type, count in edge_types.items():
        print(f"- {edge_type}: {count} edges")
    
    # Calculate network density
    density = nx.density(G)
    print(f"\nNetwork density: {density:.6f}")
    
    # Calculate average degree
    degrees = [d for _, d in G.degree()]
    avg_degree = sum(degrees) / len(degrees)
    print(f"Average degree: {avg_degree:.2f}")


In [46]:
def run_us_china_analysis(cluster_output_df, combined_cluster_edges_df, news_df, events_df):
    """Run a simplified version of the analysis workflow with your data"""
    
    # Step 1: Apply optimizations
    print("1. Applying optimizations")
    pruned_edges = prune_edges(combined_cluster_edges_df)
    topic_clusters, important_events = create_hierarchical_graph(
        cluster_output_df, 
        pruned_edges
    )
    
    # Step 2: Create the base graph
    print("2. Creating the base graph")
    G = create_integrated_knowledge_graph(
        cluster_output_df,
        pruned_edges,
        news_df,
        events_df
    )
    
    # Step 3: Create filtered visualization
    print("3. Creating filtered visualization")
    print("\nOriginal graph statistics:")
    analyze_network_statistics(G)
    
    net = create_filtered_graph(G, topic_clusters, important_events,
                              max_nodes=1000,  # Show maximum 1000 nodes
                              min_connections=2,  # Require at least 2 connections
                              show_only_important=True)  # Show only important events
    
    # Save and return
    output_file = "us_china_knowledge_graph_optimized.html"
    net.save_graph(output_file)
    
    return {
        'graph': G,
        'topic_clusters': topic_clusters,
        'important_events': important_events,
        'output_file': output_file
    }

In [47]:
outputs = run_us_china_analysis(cluster_output, combined_cluster_edges, news, events)

1. Applying optimizations


/var/folders/dd/zy6rfq2x43z3j43d1_ddscth0000gp/T/ipykernel_99769/2053828737.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  important_events = topic_clusters.groupby('topic_cluster').apply(get_important_events)


2. Creating the base graph
Adding cluster events to graph...
Adding causal edges to graph...
Adding news nodes to graph...
Extracting events from news articles...
Knowledge graph created with 39672 nodes and 8081 edges
3. Creating filtered visualization

Original graph statistics:
Network Statistics:
Total nodes: 39672
Total edges: 8081

Node Types:
- event_cluster: 31996 nodes
- news: 7674 nodes
- extracted_event: 2 nodes

Edge Types:
- causal: 359 edges
- mention: 7707 edges
- dependency: 15 edges

Network density: 0.000005
Average degree: 0.41

Filtered graph statistics:
Nodes remaining: 111 (from 39672)
Edges remaining: 128 (from 8081)
Topics represented: 10
